# Image classifier approach

This notebook describes my approach to image classification by some characteristics

## Features

### Color simmilarities

- Convert rgb values to light frequency
- plot values as curve to get color spectrum of the image
    - bin size for counting controls precision
- pick three highest peaks
- find image with the most similar peak values

### Feature similarities

- using ```cv.approxPolyDP()``` we find primitives in the image
    - [stackoverflow problem description](https://stackoverflow.com/questions/11424002/how-to-detect-simple-geometric-shapes-using-opencv)
- make list of primitives with type, average color, position and size
- compare with other images, turning on and off different similarity filters

## Similarity rating

Selecting the different aspects should give some normalized rating of how similar the images are.

## Addition
### Overall
I agree that adding different options that the user can turn on or off would make the system more flexible.

If we have time at the end, we could try building a GUI so the user can interact with these options.

It could also be useful to let the user adjust settings, like how many top colors to compare (e.g., top 1, 2, 3…), or choose between histogram-based or cluster-based approaches for color comparison.


### Some additions and thoughts:
- not sure if "light frequency" == HSV? 
- I would definetly use HSV
- we could cluster all HSV values using KMeans to find the main colors in the image. The resulting clusters would then contain both: color centers and their relative weight so that we can compare the overall color distribution in the images.



### To find similarities in where those colors are located in the image:
- we can add pixel coordinates as a feature in the KMeans clustering.
OR 
- a blurred version of the reference image and use it as a soft mask to compare how colors are spread out in the image. --> general idea of the color layout without focusing on exact color matches.
OR
- (I think you mentioned something like this before) split the image into a grid and calculate the main color for each section. -> low-resolution color map to compare with maps from other images.

In [1]:
# imports

import 

SyntaxError: Expected one or more names after 'import' (3942354367.py, line 3)